<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

  Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)

In [1]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/domenic/Documents/Spinal-bootcamp/source/load-spinal.sc"

Hierarchy
=========

## Component 

Like in VHDL and Verilog, you can define components that can be used to build a design hierarchy. However, in SpinalHDL, you don’t need to bind their ports at instantiation


In [2]:
class Sub extends Component{
  val a = in UInt(8 bits)
  val b = out UInt()    
  b :=  a
}

defined class Sub

In [3]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
    
  val u_sub = new Sub 
  u_sub.a := a
  b := u_sub.b
}
showRtl(new Top) 

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:30:54
[Progress] at 0.000 : Elaborate components
[Progress] at 0.127 : Checks and transforms
[Progress] at 0.190 : Generate Verilog
[Done] at 0.288
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
  input      [7:0]    a,
  output     [7:0]    b
);
  wire       [7:0]    u_sub_b;

  Sub u_sub (
    .a    (a[7:0]        ), //i
    .b    (u_sub_b[7:0]  )  //o
  );
  assign b = u_sub_b;

endmodule

module Sub (
  input      [7:0]    a,
  output     [7:0]    b
);

  assign b = a;

endmodule



defined class Top

### Full adder

In [4]:
class AdderCell extends Component {
  //Declaring all in/out in an io Bundle is probably a good practice
  val io = new Bundle {
    val a, b, cin = in Bool
    val sum, cout = out Bool
  }
  //Do some logic
  io.sum := io.a ^ io.b ^ io.cin
  io.cout := (io.a & io.b) | (io.a & io.cin) | (io.b & io.cin)
}

class Adder(width: Int) extends Component {
 
  //Create 2 AdderCell
  val cell0 = new AdderCell
  val cell1 = new AdderCell
  cell1.io.cin := cell0.io.cout   //Connect cout of cell0 to cin of cell1

  // Another example which create an array of ArrayCell
  val cellArray = Array.fill(width)(new AdderCell)
  cellArray(1).io.cin := cellArray(0).io.cout   //Connect cout of cell(0) to cin of cell(1)
 
}
showRtl(new Adder(8)) 

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:32:52
[Progress] at 118.258 : Elaborate components
[Progress] at 118.277 : Checks and transforms
[Progress] at 118.308 : Generate Verilog
[Warning] 50 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 118.360
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Adder
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Adder (
);
  wire                _zz_1;
  wire                _zz_2;
  wire                _zz_3;
  wire                _zz_4;
  wire                _zz_5;
  wire                _zz_6;
  wire                _zz_7;
  wire                _zz_8;
  wire                _zz_9;
  wire                _zz_10;
  wire                _zz_11;
  wire                _zz_12;
  wire                _zz_13;
  wire       

defined class AdderCell
defined class Adder

In [5]:
Range(1,9).foldLeft(0)((a,b)=>{println(s"$a-->$b");b})

0-->1
1-->2
2-->3
3-->4
4-->5
5-->6
6-->7
7-->8


res4: Int = 8

### Input / output definition
The syntax to define inputs and outputs is the following:

Syntax | Description | Return
-|-|-
in/out Bool | Create an input/output Bool | Bool
in/out Bits/UInt/SInt[(x bit)] | Create an input/output of the corresponding type | T
in/out(T) | For all other data types, you should add the brackets around it.<br> Sorry this is a Scala limitation. | T
master/slave(T) | This syntax is provided by the spinal.lib. T should extend IMasterSlave : <br>Some documentation is available here |T

In [6]:
 class Top extends Component{
  val a = slave Flow(UInt(8 bits))
  val b = master Flow(UInt(8 bits)) 
  b << a //  b <> a also ok
}
showRtl(new Top) 

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:35:04
[Progress] at 249.852 : Elaborate components
[Progress] at 249.871 : Checks and transforms
[Progress] at 249.874 : Generate Verilog
[Done] at 249.906
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
  input               a_valid,
  input      [7:0]    a_payload,
  output              b_valid,
  output     [7:0]    b_payload
);

  assign b_valid = a_valid;
  assign b_payload = a_payload;

endmodule



defined class Top

There are some rules to follow with component interconnection:

- Components can only read output and input signals of child components
- Components can read their own output port values (unlike VHDL)

### Jump wire（飞线）
If for some reason, you need to read signals from far away in the hierarchy (debug, temporal patch) 
you can do it by using the value returned by **`some.where.else.theSignal.pull()`**.

In [7]:
class xxCtrl extends Component{
  val start   = in Bool()
  val end     = out Bool()
  val counter = Reg(UInt(8 bits)) init 0
  when(start){counter.clearAll}
  .otherwise{counter := counter + 1}
  end := counter === 255
}

class xxTop extends Component{
    val start = in Bool()    
    val xx = out UInt()
    
    val ctrl = new xxCtrl    
    ctrl.start := start
    
    xx :=  ctrl.counter.pull() //Jump wire auto through IO
}
showRtl(new xxTop) 

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:37:21
[Progress] at 387.448 : Elaborate components
[Progress] at 387.463 : Checks and transforms
[Progress] at 387.483 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 387.518
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : xxTop
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module xxTop (
  input               start,
  output     [7:0]    xx,
  input               clk,
  input               reset
);
  wire                ctrl_end_1;
  wire       [7:0]    ctrl__zz_1;

  xxCtrl ctrl (
    .start    (start            ), //i
    .end_1    (ctrl_end_1       ), //o
    ._zz_1    (ctrl__zz_1[7:0]  ), //o
    .clk      (clk              ), //i
    .reset    (reset            )  //i
  );


defined class xxCtrl
defined class xxTop

### Pruned signals
SpinalHDL will never Pruned signals with names, for those resones:

- Sometime they are used for debug purposes in the wave
- Sometime, they should be part of the usefull RTL, but the user forgot to connect something to realy make them usefull. If they are removed from the netlist that's realy confusing to the user, and kind of hard to trace back where the missing connection is
- It allow to design things without having the whole thing done and look at the wave already

In [8]:
class TopLevel extends Component { 
  val notRemoved1 = UInt(8 bits)
  val notRemoved2 = UInt(8 bits) 
  Reg(UInt(8 bits)) init 0  //pruned signal without name without loads
}
showRtl(new TopLevel)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:41:07
[Progress] at 612.716 : Elaborate components
[Progress] at 612.717 : Checks and transforms
[Progress] at 612.719 : Generate Verilog
[Warning] 2 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 612.742
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : TopLevel
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module TopLevel (
);
  wire       [7:0]    notRemoved1;
  wire       [7:0]    notRemoved2;


endmodule



defined class TopLevel

### Generic(VHDL) / Parameter(Verilog)
If you want to parameterize your component, you can give parameters to the constructor of the component as follows:

In [9]:
class MyAdder(width: BitCount) extends Component {
  val io = new Bundle{
    val a,b    = in UInt(width)
    val result = out UInt(width)
  }
  io.result := io.a + io.b
}

showRtl(new MyAdder(8 bits))

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:43:17
[Progress] at 743.009 : Elaborate components
[Progress] at 743.011 : Checks and transforms
[Progress] at 743.013 : Generate Verilog
[Done] at 743.030
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : MyAdder
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module MyAdder (
  input      [7:0]    io_a,
  input      [7:0]    io_b,
  output     [7:0]    io_result
);

  assign io_result = (io_a + io_b);

endmodule



defined class MyAdder

I you have several parameters, it is a good practice to give a specific configuration class as follows:
```scala
case class MySocConfig(axiFrequency  : HertzNumber,
                       onChipRamSize : BigInt,
                       cpu           : RiscCoreConfig,
                       iCache        : InstructionCacheConfig)

class MySoc(config: MySocConfig) extends Component {
    ...
}
```

## Function
The ways you can use Scala functions to generate hardware are radically different than VHDL/Verilog for many reasons:

- You can instantiate registers, combinatorial logic and components inside them.

- You don’t have to play with process/@always that limit the scope of assignment of signals

- Everything is passed by reference, which allows easy manipulation.
For example you can give a bus to a function as an argument, then the function can internaly read/write to it.
You can also return a Component, a Bus, or anything else from scala and the scala world.

### Component whith function

In [10]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
  val c = out UInt(8 bits) 
    
  def pass(x: UInt, n : Int) = {
      val ret = UInt(n bits)
          ret := x 
      ret 
  }
    
  def pass2(x: UInt) = {
      class Fix(n: Int) extends Component {
          val a = in UInt()
          val b = out  UInt() 
          b := pass(in(a), n)
      }
      val res = new Fix(x.getWidth)
      res.a := x
      res.b
  }
    b := pass(a,8)
    c := pass2(a)
}
showRtl(new Top)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:46:32
[Progress] at 938.265 : Elaborate components
[Progress] at 938.266 : Checks and transforms
[Progress] at 938.271 : Generate Verilog
[Done] at 938.295
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
  input      [7:0]    a,
  output     [7:0]    b,
  output     [7:0]    c
);
  wire       [7:0]    fix_1_b;

  Fix fix_1 (
    .a    (a[7:0]        ), //i
    .b    (fix_1_b[7:0]  )  //o
  );
  assign b = a;
  assign c = fix_1_b;

endmodule

module Fix (
  input      [7:0]    a,
  output     [7:0]    b
);

  assign b = a;

endmodule



defined class Top

### RGB to gray
For example if you want to convert a Red/Green/Blue color into greyscale by using coefficients, you can use functions to apply them:

In [11]:
class Top extends Component{
    // Input RGB color
val r, g, b = UInt(8 bits)

// Define a function to multiply a UInt by a scala Float value.
def coef(value: UInt, by: Float): UInt = (value * U((255*by).toInt, 8 bits) >> 8)

// Calculate the gray level
val gray = coef(r, 0.3f) + coef(g, 0.4f) + coef(b, 0.3f)
}
showRtl(new Top) 

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:48:28
[Progress] at 1054.348 : Elaborate components
[Progress] at 1054.349 : Checks and transforms
[Progress] at 1054.354 : Generate Verilog
[Warning] 4 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1054.377
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
);
  wire       [7:0]    _zz_1;
  wire       [7:0]    _zz_2;
  wire       [15:0]   _zz_3;
  wire       [7:0]    _zz_4;
  wire       [15:0]   _zz_5;
  wire       [7:0]    _zz_6;
  wire       [15:0]   _zz_7;
  wire       [7:0]    r;
  wire       [7:0]    g;
  wire       [7:0]    b;
  wire       [7:0]    gray;

  assign _zz_1 = (_zz_2 + _zz_4);
  assign _zz_2 = (_zz_3 >>> 8);
  assign _zz_3 = (r * 

defined class Top

### Valid Ready Payload bus
For instance if you define a simple Valid Ready Payload bus, you can then define some useful functions inside of it.

In [12]:
case class MyBus(payloadWidth: Int) extends Bundle with IMasterSlave {
  val valid   = Bool
  val ready   = Bool
  val payload = Bits(payloadWidth bits)

  // define the direction of the data in a master mode
  override def asMaster(): Unit = {
    out(valid, payload)
    in(ready)
  }

  // Connect that to this
  def <<(that: MyBus): Unit = {
    this.valid   := that.valid
    that.ready   := this.ready
    this.payload := that.payload
  }

  // Connect this to the FIFO input, return the fifo output
  def queue(size: Int): MyBus = {
    val fifo = new MyBusFifo(payloadWidth, size)
    fifo.io.push << this
    return fifo.io.pop
  }
}

class MyBusFifo(payloadWidth: Int, depth: Int) extends Component {
  val io = new Bundle {
    val push = slave(MyBus(payloadWidth))
    val pop  = master(MyBus(payloadWidth))
  }
 io.pop <> io.push
}

class Top extends Component {
  val io = new Bundle {
    val idata = slave(MyBus(8))
    val odata  = master(MyBus(8))
  }
  io.odata << io.idata.queue(32)
}
showRtl(new Top)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:50:52
[Progress] at 1198.338 : Elaborate components
[Progress] at 1198.351 : Checks and transforms
[Progress] at 1198.355 : Generate Verilog
[Done] at 1198.373
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
  input               io_idata_valid,
  output              io_idata_ready,
  input      [7:0]    io_idata_payload,
  output              io_odata_valid,
  input               io_odata_ready,
  output     [7:0]    io_odata_payload
);
  wire                myBusFifo_1_io_push_ready;
  wire                myBusFifo_1_io_pop_valid;
  wire       [7:0]    myBusFifo_1_io_pop_payload;

  MyBusFifo myBusFifo_1 (
    .io_push_valid      (io_idata_valid                   ), //i
    .io_push_ready      (myBusF

defined class MyBus
defined class MyBusFifo
defined class Top

## Area
Sometimes, creating a Component to define some logic is overkill because you:   

- Need to define all construction parameters and IO (verbosity, duplication)
- Split your code (more than needed)

For this kind of case you can use an Area to define a group of signals/logic.

In [13]:
class UartCtrl extends Component {
  
  val timer = new Area {
    val counter = Reg(UInt(8 bit))
    val tick = counter === 0
    counter := counter - 1
    when(tick) {
      counter := 100
    }
  }

  val tickCounter = new Area {
    val value = Reg(UInt(3 bit))
    val reset = False
    when(timer.tick) {          // Refer to the tick from timer area
      value := value + 1
    }
    when(reset) {
      value := 0
    }
  }

  val stateMachine = new Area {
   
  }
}
showRtl(new UartCtrl)

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:52:55
[Progress] at 1320.864 : Elaborate components
[Progress] at 1320.873 : Checks and transforms
[Progress] at 1320.879 : Generate Verilog
[Warning] 6 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1320.898
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : UartCtrl
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module UartCtrl (
  input               clk,
  input               reset
);
  reg        [7:0]    timer_counter;
  wire                timer_tick;
  reg        [2:0]    tickCounter_value;
  wire                tickCounter_reset;

  assign timer_tick = (timer_counter == 8'h0);
  assign tickCounter_reset = 1'b0;
  always @ (posedge clk) begin
    timer_counter <= (timer_counter - 8'h01);
    if(timer_tick

defined class UartCtrl

In VHDL and Verilog, sometimes prefixes are used to separate variables into logical sections.  
It is suggested that you use `Area` instead of this in SpinalHDL.  

[ClockingArea](https://spinalhdl.github.io/SpinalDoc-RTD/SpinalHDL/Structuring/clock_domain.html#clock-domain) is a special kind of `Area` that allows you to define chunks of hardware which use a given `ClockDomain`

## Example: Parameter Pipline AdderTree 
**requirement**:

- VectorSize Configurable
- GroupSize Configurable, auto Group
- DataWidth Configurable

![adderTree1](./source/addertree1.png)

### 1st: creat Sum-Component which adder all Vec input together in one cycle with Register out

In [14]:
class Sum(diw: Int, size: Int, stage: Int) extends Component{
    this.setDefinitionName(s"sum_stage${stage}_n${size}_w${diw}")
    val dow = diw + log2Up(size)

    val io = new Bundle{
      val nets = slave Flow(Vec(SInt(diw bits), size))
      val sum  = out(SInt(dow bits)).setAsReg()
    }

    when(io.nets.valid){
      io.sum := io.nets.payload
        .map(_.resize(dow bits))
        .reduce(_ + _)
    }
}
showRtl(new Sum(8, 2, 0))
showRtl(new Sum(8, 3, 0))

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:56:47
[Progress] at 1553.007 : Elaborate components
[Progress] at 1553.023 : Checks and transforms
[Progress] at 1553.026 : Generate Verilog
[Done] at 1553.052
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : sum_stage0_n2_w8
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module sum_stage0_n2_w8 (
  input               io_nets_valid,
  input      [7:0]    io_nets_payload_0,
  input      [7:0]    io_nets_payload_1,
  output reg [8:0]    io_sum,
  input               clk,
  input               reset
);
  wire       [8:0]    _zz_1;
  wire       [8:0]    _zz_2;

  assign _zz_1 = {{1{io_nets_payload_0[7]}}, io_nets_payload_0};
  assign _zz_2 = {{1{io_nets_payload_1[7]}}, io_nets_payload_1};
  always @ (posedge clk) begin
    if(io_nets_valid)begin
      io_sum <= ($signed

defined class Sum

### 2st: Creating recursive functions
Auto Sum together with pipeline divide by Group-size

In [15]:
private def sumAdd(nets: Flow[Vec[SInt]], stage: Int): Sum = {
    val uSum = new Sum(nets.payload.head.getWidth, nets.payload.size, stage)
    uSum.io.nets.valid   := nets.valid
    uSum.io.nets.payload := nets.payload.resized
    uSum
  }

def pipeTree(nets: Flow[Vec[SInt]], groupSize: Int , stage: Int = 0): (List[Sum], Int) = {

    val nextstage = stage + 1

    if (nets.payload.size <= groupSize) {
      (List(sumAdd(nets, nextstage)), nextstage)
    } else {
      val grpNum = scala.math.ceil(nets.payload.size.toDouble / groupSize).toInt
      val nextStage = (0 until grpNum).toList
        .map(i => nets.payload.drop(i * groupSize).take(groupSize))
        .map{ grouped =>
          val groupednets = Flow(Vec(SInt(grouped.head.getWidth bits), grouped.size))
          groupednets.valid   := nets.valid
          groupednets.payload := Vec(grouped)
          sumAdd(groupednets, nextstage)
        }
      val ret = Flow(Vec(SInt(nextStage.head.io.sum.getWidth bits), nextStage.size))
      ret.valid   := RegNext(nets.valid, init = False)
      ret.payload := Vec(nextStage.map(_.io.sum)).resized
      pipeTree(ret, groupSize, nextstage)
    }
}

defined function pipeTree

### 3st[option]: given adder tree with a component Top

In [16]:
class AdderTree(diw: Int, size: Int, groupSize: Int) extends Component{

    val io_nets = slave Flow(Vec(SInt(diw bits), size))

    val (sum, stage) = pipeTree(io_nets, groupSize, 0)

    this.setDefinitionName(s"adderTree_n${size}_g${groupSize}_dly${stage}")

    def Latency: Int = stage

    def dow: Int = diw + log2Up(groupSize) * stage

    val io_sum  = master Flow(SInt(sum.head.io.sum.getWidth bits))

    io_sum.payload := sum.head.io.sum
    io_sum.valid   := RegNext(sum.head.io.nets.valid, init = False)
}

defined class AdderTree

### 4st[option]: Provides a simple common function

In [17]:
object AdderTree {
  def apply(nets: Flow[Vec[SInt]], addCellSize: Int): AdderTree = {
    val uAdderTree = new AdderTree(nets.payload.head.getWidth, nets.payload.size, addCellSize)
    uAdderTree.io_nets := nets
    uAdderTree
  }

  def apply(nets: Vec[SInt], addCellSize: Int): AdderTree = {
    val uAdderTree = new AdderTree(nets.head.getWidth, nets.size, addCellSize)
    uAdderTree.io_nets.payload := nets
    uAdderTree.io_nets.valid   := True
    uAdderTree
  }
}

defined object AdderTree

### Usage:
```scala
class Top extends Component{
  val nets = Flow(Vec(SInt(8 bits), 23))
  AdderTree(nets, 2)//group size = 2
}
showRtl(new Top)
```
this will generator diagram [below](http://localhost:8888/notebooks/2.6-Spinal-core-Component-Function-Area.ipynb#Example:-Parameter-Pipline-AdderTree) 

In [18]:
class Top extends Component{
    val nets = Flow(Vec(SInt(8 bits), 23))
    AdderTree(nets, addCellSize = 4)//group size = 4
}

showRtl(new Top)
// we change the addCellSize/groupSize = 4 , then got following diagram  

[Runtime] SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
[Runtime] JVM max memory : 1820.5MiB
[Runtime] Current date : 2020.09.24 08:58:30
[Progress] at 1656.475 : Elaborate components
[Progress] at 1656.513 : Checks and transforms
[Progress] at 1656.553 : Generate Verilog
[Warning] 123 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 1656.589
// Generator : SpinalHDL v1.4.1    git head : d1b4746673438bc5f242515335278fa39a666c38
// Component : Top
// Git hash  : cd00610a9b5416861f9d7ebad6d751cddb5209a2



module Top (
  input               clk,
  input               reset
);
  wire                adderTree_io_sum_valid;
  wire       [12:0]   adderTree_io_sum_payload;
  wire                nets_valid;
  wire       [7:0]    nets_payload_0;
  wire       [7:0]    nets_payload_1;
  wire       [7:0]    nets_payload_2;
  wire       [7:0]    nets_payload_3;
  wire       [7:0]    nets_payload_4;
  wire       [7:0]  

    .clk                  (clk                 ), //i
    .reset                (reset               )  //i
  );
  sum_stage2_n2_w10 sum_7 (
    .io_nets_valid        (_zz_1               ), //i
    .io_nets_payload_0    (sum_4_io_sum[9:0]   ), //i
    .io_nets_payload_1    (sum_5_io_sum[9:0]   ), //i
    .io_sum               (sum_7_io_sum[10:0]  ), //o
    .clk                  (clk                 ), //i
    .reset                (reset               )  //i
  );
  sum_stage3_n2_w12 sum_0 (
    .io_nets_valid        (_zz_1_regNext       ), //i
    .io_nets_payload_0    (sum_6_io_sum[11:0]  ), //i
    .io_nets_payload_1    (_zz_2[11:0]         ), //i
    .io_sum               (sum_0_io_sum[12:0]  ), //o
    .clk                  (clk                 ), //i
    .reset                (reset               )  //i
  );
  assign _zz_1 = io_nets_valid_regNext;
  assign _zz_2 = {{1{sum_7_io_sum[10]}}, sum_7_io_sum};
  assign io_sum_payload = sum_0_io_sum;
  assign io_sum_valid = sum_0_io_nets

defined class Top

![adderTree2](./source/addertree2.png)